In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
import lxml
import lxml.html.soupparser
import re
#from IPython.display import HTML

In [2]:
#webdriver_path = '/Users/zzh/Desktop/chromedriver'
#webdriver_path = r'C:\Users\spy61\Desktop\webscraper\chromedriver.exe'  # your filepath here
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [3]:
driver = webdriver.Chrome(executable_path=webdriver_path, options=options)
threaddriver = webdriver.Chrome(executable_path=webdriver_path, options=options)
# Download html with Selenium
baseUrl = 'https://community.byte.co'
pageUrl = baseUrl + '/latest'
driver.get(pageUrl)

In [4]:
pageUrl

'https://community.byte.co/latest'

In [5]:
# Parse html with beautifulSoup
soup = BeautifulSoup(driver.page_source,'lxml')

In [6]:
post_links = []

# get the initial height of web page
lastHeight = driver.execute_script("return document.body.scrollHeight")

# the while loop is to check whether you reach the very bottom of the webpage
while True:
    # tell selenium to scroll down to the current bottom of webpage
    driver.execute_script("window.scrollTo(0, 1080)")
    # change number here to decide how much to go through
    
    # To the bottom
    # driver.execute_script("window.scrollTo(arguments[0], document.body.scrollHeight);",lastHeight)
    
    # Parse html with beautifulSoup
    soup = BeautifulSoup(driver.page_source,'lxml')
    # locate the html section with posts
    body = soup.find('body')
    main_outlet = body.find('div',{'class':"container list-container"})

    # get all hyperlinks of post
    all_a = main_outlet.findAll('a',{'class':'title raw-link raw-topic-link'})
    for a in all_a:
        post_link = a.get('href')
        post_links.append(post_link)
    
    # calculate the new height of webpage
    newHeight = driver.execute_script("return document.body.scrollHeight")
    
    if newHeight == lastHeight:
        break
    
    lastHeight = newHeight

In [7]:
post_links

['/t/tip-external-editing-for-near-perfect-audio-loops/39519',
 '/t/byte-con-locations/57032',
 '/t/how-to-add-link-in-bio/57236',
 '/t/featured-sounds/57221',
 '/t/underrated-byters/52553',
 '/t/glow-in-the-dark-bytes/57237',
 '/t/creating-content-while-depressed/57047',
 '/t/will-we-have-the-opportunity-to-be-verified/14008',
 '/t/seeking-feedback-profile/57234',
 '/t/gen-z-on-byte/57200',
 '/t/tiktokers-on-byte/57217',
 '/t/countries-that-youve-been-to/33659',
 '/t/im-surprised/57230',
 '/t/college-freshman-advice/57079',
 '/t/favorite-pop-songs-of-2020/57203',
 '/t/3-isnt-for-everybody/57227',
 '/t/hi-im-dylan-tate/53903',
 '/t/the-greatest-inventions-from-your-country/54042',
 '/t/our-basement/56426',
 '/t/petitions-you-should-sign/56477',
 '/t/the-byte-house/57157',
 '/t/tiktok-is-officially-getting-banned-in-the-us/56928',
 '/t/write-a-haiku-for-a-movie-and-you-have-to-guess-it/27042',
 '/t/my-first-hate-comment/56790',
 '/t/to-my-og-s/57143',
 '/t/someone-be-proud-pls/57133',
 

In [8]:
# titles = soup.find_all('span', {'class':'link-top-line'})
# urls = []
# for title in titles:
#     t = title.find('a', {'class':'title raw-link raw-topic-link'})
#     urls.append(t['href'])
    

In [9]:
# urls

In [10]:
titles = []
cats = []
names = []
likes = []
posts = []
dates = []
for url in post_links:
    threadUrl = baseUrl + url
    threaddriver.get(threadUrl)
    soup = BeautifulSoup(threaddriver.page_source, 'lxml')
    
    # get title
    title = soup.find('a', {'class': 'fancy-title'}).getText().strip()
    # get category
    cat_class = soup.find('div', {'class': 'title-wrapper'})
    cat = cat_class.find('span', {'class': 'category-name'}).getText()
    
    #use a regular expression to match all posts in the thread, including the opening post
    for reply in soup.findAll('div',{'class':re.compile('^topic-post clearfix')}): 
        names.append(reply.find('div',{'class':'names trigger-user-card'}).getText())
        posts.append(reply.find('div',{'class':'cooked'}).getText())
        dates.append(reply.find('div',{'class':'post-info post-date'}).getText())
        likes.append(reply.find('div',{'class':'double-button'}).getText())
        titles.append(title)
        cats.append(cat)

In [11]:
posts_dict = {}
posts_dict['title'] = titles
posts_dict['catagory'] = cats
posts_dict['name'] = names
posts_dict['likes'] = likes
posts_dict['content'] = posts
posts_dict['date'] = dates 
posts_dict

{'title': ['Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfect audio loops',
  'Tip: External editing for near-perfe

In [12]:
df = pd.DataFrame.from_dict(posts_dict,orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,424,425,426,427,428,429,430,431,432,433
title,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,Tip: External editing for near-perfect audio l...,...,Text feature glitch (my texts were out of place),Text feature glitch (my texts were out of place),Text feature glitch (my texts were out of place),How does byte handle ads?,How does byte handle ads?,How does byte handle ads?,How does byte handle ads?,How does byte handle ads?,How does byte handle ads?,How does byte handle ads?
catagory,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,...,Bug Reports,Bug Reports,Bug Reports,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion,General Discussion
name,domstaff ⭐️,HeyWhatsNewExpert,carkEmpathetic,Pgozur,Rafiki,freeze,AlmoExpert,Tatyana,domstaff ⭐️,ShonziThoAdmired,...,mehamberLicensed,LeaElizabeth,oddbydefault,Kerbob97,Pac,dagdragon,Kerbob97,Kerbob97,LeaElizabeth,LeaElizabeth
likes,114,8,9,6,18,5,3,3,5,6,...,2,1,,3,1,2,1,1,1,1
content,For important (and mostly unavoidable) reasons...,"Very helpful tips, thanks.","thank you dom, very cool!",Thanks for this!,thanks for the tip \n\n48B616CA-0E92-44A9-9937...,I’m curious if following this procedure will a...,Thanks for this Dom,editing tips \nphannies of 2018 will understan...,it should,videos seem to loop differently on android and...,...,@dom This happens to me too. I have the andro...,\n\n\n\nIn app text feature preview displaying...,"No, I think it’s a completely different one. I...","In general on most platforms, if you’re doing ...","cookies looks yummy, im curious as well (but ...",I’ve never thought about the logistics of this...,They were amazing. Great flavor and texture a...,"Yeah, the last thing I want to do is come acro...",I think it depends if the company that gave yo...,"You def dont seem spammy with it either, so be..."
date,Oct '19,Oct '19,Oct '19,Oct '19,Oct '19,Dec '19,Dec '19,Dec '19,Dec '19,Dec '19,...,22h,21h,21h,1d,1d,1d,1d,1d,21h,21h


In [13]:
df.to_csv('web_scraping_data.csv')